#Testing the coefficients in a real simulation where the feedback is generated by the readout.

In [1]:
# Makes possible to show the output from matplotlib inline
%matplotlib inline
import matplotlib.pyplot as plt

# Makes the figures in the PNG format:
# For more information see %config InlineBackend
%config InlineBackend.figure_formats=set([u'png'])

# plt.rcParams['figure.figsize'] = 5, 10

import numpy
import sys
import os

import save_load_file as slf

In [18]:
import matplotlib 

# font = {'family' : 'normal',
#         'weight' : 'bold',
#         'size'   : 22}

font = {'weight' : 'normal',
        'size'   : 15}

matplotlib.rc('font', **font)

# matplotlib.rcdefaults() # restores to the default values

# http://matplotlib.org/api/matplotlib_configuration_api.html#matplotlib.rc

# Main functions to the 2 joint arm simulation

In [19]:
from simulation_2DoF_Arm_physics import *
reload(sys.modules['simulation_2DoF_Arm_physics']) # Makes sure the interpreter is going to reload the module

<module 'simulation_2DoF_Arm_physics' from 'simulation_2DoF_Arm_physics.pyc'>

#Start of the simulation!

In [20]:
trajectories = [[[0.75,0.25],[0.0,0.5]],\
                [[0.25,0.60],[-0.25,0.60]],\
                [[-0.10,0.75],[-0.10,0.25]],\
                [[-0.75,0.50],[-0.40,0.00]]]

In [21]:
# First the arm parametres according to Joshi/Maass paper
aparams = {
    'l1' : 0.5, # metres
    'l2' : 0.5,
    'lc1' : 0.25,
    'lc2' : 0.25,
    'm1' : 1.0, # kg
    'm2' : 1.0,
    'i1' : 0.03, # kg*m*m
    'i2' : 0.03
}

In [22]:
# Total time spent during the movement (in seconds)
MT = 0.5
# Simulation time step (in seconds)
time_step = 2/1000.0
# Number of neurons at each input layer (defines the resolution of the system)
Ninput = 50

In [23]:
t_mov=numpy.arange(0, MT, time_step) # t starts in 0s and steps time_step(s) until reaches MT(s)

In [24]:
sim_set = "set_01"
total_trials = 50

base_dir = "2DofArm_simulation_data"

# Reads all the experiments to check which range of torques is necessary to control the arm
# These values are based ONLY in the 4 trajectories used!
torques_all_1 = numpy.array([]);
torques_all_2 = numpy.array([]);
number_of_trajectories = len(trajectories)
for i in range(1,number_of_trajectories+1):
    states,torques = slf.load_from_file_gz("./"+base_dir+"/"+sim_set+"/States_Torques_movement"+str(i)+".gzpickle")
    torques_all_1 = numpy.concatenate((torques_all_1,torques[:,0])) # torques joint 1
    torques_all_2 = numpy.concatenate((torques_all_2,torques[:,1])) # torques joint 2

In [25]:
# Creates the 50 possible discrete values of torques for each joint (probably will be different)
torques_values_1=numpy.linspace(torques_all_1.min(),torques_all_1.max(),num=50)
torques_values_2=numpy.linspace(torques_all_2.min(),torques_all_2.max(),num=50)

In [26]:
# Creates the 50 possible discrete values of angles for each joint
teta1_values=numpy.linspace(-numpy.pi/6,numpy.pi,num=50)
teta2_values=numpy.linspace(0,numpy.pi,num=50)

In [27]:
# These are the 50 possible values that the input neurons will represent (x,y) 
x_values=numpy.linspace(-1,1,num=50)
y_values=numpy.linspace(-1,1,num=50)

In [28]:
def generates_lsm_start(tji):
    xstart,ystart = trajectories[tji-1][0]
    xdest,ydest = trajectories[tji-1][1]


    # The original system, proposed by Joshi/Maass 2006 starts without any information about the current
    # position because the proprioceptive feedback comes only after a time delay.

    # Now I will add some noise
    # noise_sd = 0.1
    # xstart,ystart = xstart+numpy.random.normal(loc=0,scale=noise_sd),ystart+numpy.random.normal(loc=0,scale=noise_sd)
    # xdest,ydest = xdest+numpy.random.normal(loc=0,scale=noise_sd),ydest+numpy.random.normal(loc=0,scale=noise_sd)

    # print "Original:", trajectories[tji-1][0],trajectories[tji-1][1]
    # print "Noisy:",[xstart,ystart],[xdest,ydest]


    # Indexes of the normalized initial position values to use in the LSM simulation
    xstart_idx =  abs(x_values-xstart).argmin()
    ystart_idx =  abs(y_values-ystart).argmin()

    # Indexes of the normalized final position values to use in the LSM simulation
    xdest_idx =  abs(x_values-xdest).argmin()
    ydest_idx =  abs(y_values-ydest).argmin()

    (xstart_idx,ystart_idx),(xdest_idx,ydest_idx)


    # The input (one big neurongroup with 300 neurons) will be divided like this:
    # 6 groups of 50 neurons.
    # - Group 1: xdest => offset:0
    # - Group 2: ydest => offset:50
    # - Group 3: teta1 => offset:100
    # - Group 4: teta2 => offset:150
    # - Group 5: tau1  => offset:200
    # - Group 6: tau2  => offset:250

    ############# Using ONLY the initial (start) position
    # x_lsm = xstart_idx + 0
    # y_lsm = ystart_idx + 50

    ############# Using ONLY the final (dest) position
    x_lsm = xdest_idx + 0
    y_lsm = ydest_idx + 50
    return x_lsm,y_lsm,xstart,ystart,xdest,ydest

## Measuring the difference between the generated curves
$MeanDistance=\frac{1}{N} \sum \limits_{n=1}^{N}  \lVert P_{Ideal}(n)-P_{Test}(n) \rVert$

$TotalLength=\sum \limits_{n=1}^{N}  \lVert P_{Ideal}(n)-P_{Ideal}(n-1) \rVert$

$NCE=\frac{MeanDistance}{TotalLength}$

In [29]:
def MY_METRIC(vec_orig,vec):
    
    # First I calculate the mean distance between all the (x,y) points
    mean_distance=(numpy.sqrt(((vec_orig-vec)**2).sum(axis=0).mean()))
    
    # Then the line integral (the length of the curve)
    line_integral=(numpy.sqrt(((vec_orig.T[1:]-vec_orig.T[:-1])**2).sum(axis=1))).sum()

    return mean_distance/line_integral

In [30]:
# Calculates the metric in relation to the TORQUE CURVES and verifies if all the simulations were generated
error_list = []
for tji in range(1,5):
    no_error=True
    METRIC = numpy.zeros((3,3,total_trials,2),dtype=float)
    
    for noiselevel in range(3):
        for decimationlevel in range(3):
            for run_number in range(1,total_trials+1):
                try:    
                    x_lsm,y_lsm,xstart,ystart,xdest,ydest = generates_lsm_start(tji)

                    torques = slf.load_from_file("./"+base_dir+"/"+sim_set+"/torques_"+str(tji)+"_noise_level_"+str(noiselevel)+"_killed_neurons_"+str(decimationlevel)+"_run_"+str(run_number)+".pickles")

                    torques_original = numpy.array([torques_all_1[(tji-1)*250:(tji)*250],torques_all_2[(tji-1)*250:(tji)*250]])

                    METRIC[noiselevel,decimationlevel,run_number-1,:]=MY_METRIC(numpy.array([numpy.arange(0,250,dtype=float),torques_original[0,:]]),numpy.array([numpy.arange(0,250,dtype=float),torques[:,0]])),\
                    MY_METRIC(numpy.array([numpy.arange(0,250,dtype=float),torques_original[1,:]]),numpy.array([numpy.arange(0,250,dtype=float),torques[:,1]]))
                except IOError as e:
                    error_list.append([tji,decimationlevel,noiselevel,run_number])
                    print "ERROR:",len(error_list)," - ",tji,decimationlevel,noiselevel,run_number
#                     print "Description: {1}".format(e.errno, e.strerror)
                    no_error=False
    if no_error:
        slf.save_to_file(METRIC,"./"+base_dir+"/"+sim_set+"/metric_torques_trajectory_"+str(tji)+".pickles")
slf.save_to_file(numpy.array(error_list),"./"+base_dir+"/"+sim_set+"/error_list.pickles")

In [31]:
tji=1
METRIC=slf.load_from_file("./"+base_dir+"/"+sim_set+"/metric_torques_trajectory_"+str(tji)+".pickles")
METRIC[0,0,4]

In [32]:
# Calculates the metric in relation to the TRAJECTORIES!

for tji in range(1,5):
    METRIC = numpy.zeros((3,3,total_trials,1),dtype=float)
    for noiselevel in range(3):
        for decimationlevel in range(3):
            x_lsm,y_lsm,xstart,ystart,xdest,ydest = generates_lsm_start(tji)
            
            t_mov=numpy.arange(0, .5, 2.0/1000) # t starts in 0s and steps time_step(s) until reaches MT(s)

            # Generates the movements according to:
            # Flash, Tamar, and Neville Hogan. 1985
            H_path=cartesian_movement_generation_training(xstart, ystart, xdest, ydest, .5, t_mov)

            metric_traj = []
            
            for run_number in range(1,total_trials+1):

                teta = slf.load_from_file("./"+base_dir+"/"+sim_set+"/teta_"+str(tji)+"_noise_level_"+str(noiselevel)+"_killed_neurons_"+str(decimationlevel)+"_run_"+str(run_number)+".pickles")
                H,E = joints_to_hand(teta,aparams)
                METRIC[noiselevel,decimationlevel,run_number-1]= MY_METRIC(numpy.array([H_path[:,0],H_path[:,1]]),numpy.array([H[:,0][:len(H_path[:,0])],H[:,1][:len(H_path[:,0])]]))

    slf.save_to_file(METRIC,"./"+base_dir+"/"+sim_set+"/metric_trajectory_trajectory_"+str(tji)+".pickles")

In [33]:
tji=1
METRIC=slf.load_from_file("./"+base_dir+"/"+sim_set+"/metric_trajectory_trajectory_"+str(tji)+".pickles")
METRIC[0,0,0]